Comparing the DG method with the Trefftz-DG method on the anisotropic mesh in 1+1 dimensions. In this case, both, Trefftz and L2 space are constructed from a space for the second order system. Since the Hessian for the L2 space is needed the following lines have to be added in the constructor for the l2 space in l2hofespace.cpp

    switch (ma->GetDimension())
    {
    case 1:
        additional_evaluators.Set ("hesse", make_shared<T_DifferentialOperator<DiffOpHesse<1>>> ());
        break;
    case 2:
        additional_evaluators.Set ("hesse", make_shared<T_DifferentialOperator<DiffOpHesse<2>>> ());
        break;
    case 3:
        additional_evaluators.Set ("hesse", make_shared<T_DifferentialOperator<DiffOpHesse<3>>> ());
        additional_evaluators.Set ("hesseboundary", make_shared<T_DifferentialOperator<DiffOpHesseBoundary<3>>> ());
        break;
    default:
        ;
    }

## Define Functions

In [2]:
import sys
sys.path.append("..")
from ngsolve import *
SetHeapSize(100*1000*1000)

from trefftzngs import *
from DGeq import *
from prodmesh import *
import time
from netgen.geom2d import unit_square

In [3]:
def SolveWaveeq(fes,fullsys,mesh):
    [a,f] = DGeqsys(fes,truesol,v0,sig0,c,v0,fullsys)
    [gfu, cond] = DGsolve(fes,a,f)
    
    L2error = sqrt(Integrate((truesol - gfu)*(truesol - gfu), mesh))
    
    gradtruesol = CoefficientFunction((-sig0,v0))
    sH1error = sqrt(Integrate((gradtruesol - grad(gfu))*(gradtruesol - grad(gfu)), mesh))
    
    dof=fes.ndof/mesh.ne
    
    return [dof,cond,L2error,sH1error]

## Run comparison

In [ ]:
c = 1
t_steps = 1/4
nt_steps = 1
order = 8
k = 1

truesol =  sin( k*(c*z+x+y) )#exp(-pow(c*x+y,2)))#
v0 = c*k*cos(k*(c*z+x+y))#grad(U0)[0]
sig0 = CoefficientFunction( (-k*cos(k*(c*z+x+y)),-k*cos(k*(c*z+x+y))) )#-grad(U0)[1]

ngmeshbase = unit_square.GenerateMesh(maxh = t_steps)
mesh = ProdMesh(ngmeshbase,t_steps)

solution = []

for ordr in range(3,order):
    print("run order: " + str(ordr))

    #btype = 0
    #fes = L2(mesh, order=ordr, dgjumps=True)
    #[dof,cond,L2error,sH1error] = SolveWaveeq(fes, True, mesh)
    #solution.append([btype, ordr, dof,cond,L2error,sH1error])
    #print("btype: " + 'L2' + " dof: " + str(dof) + " cond: " + str(cond) + " L2error: " + str(L2error) + " H1error: "+ str(sH1error))

    btype = 1
    fes = FESpace("trefftzfespace", mesh, order = ordr, wavespeed = c, dgjumps=True, basistype = 0)
    [dof,cond,L2error,sH1error] = SolveWaveeq(fes, False, mesh)
    solution.append([btype,ordr,dof,cond,L2error,sH1error])
    print("btype: " + 'Trefftz' + " dof: " + str(dof) + " cond: " + str(cond) + " L2error: " + str(L2error) + " H1error: "+ str(sH1error))

run order: 3
btype: Trefftz dof: 16.0 cond: 1584.4696238693991 L2error: 0.004757104164565725 H1error: 0.05277760239960372
run order: 4
btype: Trefftz dof: 25.0 cond: 23914.147436236657 L2error: 0.004729417833582555 H1error: 0.05310063181632405
run order: 5
btype: Trefftz dof: 36.0 cond: 574414.5450236923 L2error: 0.00472882903992373 H1error: 0.05331939366155698
run order: 6
btype: Trefftz dof: 49.0 cond: 14459534.636063814 L2error: 0.004730350533519725 H1error: 0.053360192935537724
run order: 7


## Draw Comparison

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

label = ['','order','dof','condition', 'L2error', 'semi-H1error']
sol = np.array(solution)

for yaxis in [3,4,5]:
    for xaxis in [1,2]:
        fig = plt.figure()
        ax = fig.gca()
        plt.semilogy(sol[sol[:,0]==0,xaxis], sol[sol[:,0]==0,yaxis], '-o', label="L2 a,b=0")
        plt.semilogy(sol[sol[:,0]==1,xaxis], sol[sol[:,0]==1,yaxis], '--x', label="Trefftz a,b=0")
        ax.xaxis.set_major_locator(MaxNLocator(integer=True))
        plt.title('Wavespeed: ' + str(c) + " Meshsize: " + str("?"))
        plt.legend()
        plt.ylabel(label[yaxis])
        plt.xlabel(label[xaxis])
        #plt.savefig("results/pvtv3_"+label[yaxis]+label[xaxis]+".png")
        plt.show()